In [ ]:
async def scrape_link(context, href, max_retries=3):
    page = await context.new_page()
    attempt = 0
    while attempt < max_retries:
        try:
            await page.goto(href, timeout=60000, wait_until="load")
            await load_all_items(page, '//*[@id="facet-browse"]/section/div[3]/div[2]/div[1]/div/div')

            item_data = []

            outer_base_xpath = '//*[@id="facet-browse"]/section/div[3]/div[1]/div[5]'
            # await page.locator(f'xpath={outer_base_xpath}').wait_for(state="visible", timeout=5000)
            outer_divs = page.locator(f'xpath={outer_base_xpath}/div')
            outer_count = await outer_divs.count()

            print(f"Found {outer_count} outer divs")

            for i in range(1, outer_count + 1):

                inner_path = f'{outer_base_xpath}/div[{i}]'
                # await page.locator(f'xpath={inner_path}').wait_for(state="visible", timeout=5000)
                inner_divs = page.locator(f'xpath={inner_path}/div')
                inner_count = await inner_divs.count()

                print(f"  Outer div {i} has {inner_count} inner divs (items)")

                for j in range(1, inner_count + 1):
                    try:
                        item_xpath = f'{outer_base_xpath}/div[{i}]/div[{j}]/div/div[2]/span[2]/a/h4'
                        item_name = await page.locator(f'xpath={item_xpath}').inner_text()
                        item_data.append({"group": i, "index": j, "name": item_name})
                    except Exception as e:
                        print(f"    Failed to get phone at outer {i}, inner {j}: {e}")
                        continue

            return {
                "href": href,
                "title": f"{len(item_data)} phones found",
                "phones": item_data
            }

        except Exception as e:
            attempt += 1
            print(f"Attempt {attempt} failed for {href}: {e}")
            await asyncio.sleep(5)
        finally:
            await page.close()
    return {"href": href, "title": "Error after retries", "phones": []}




async def test():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        # Step 1: Open main page
        await page.goto("https://shop.alphacomm.com/", wait_until="load")

        # Step 2: Log in
        await page.locator('xpath=//*[@id="login-email"]').fill("Somaan@xclusivetradinginc.com")
        await page.locator('xpath=//*[@id="login-password"]').fill("Metro57398XTI$")
        await page.locator('xpath=//*[@id="LoginRegister.View"]/section/div[2]/div/div/form/fieldset/div[6]/button').click()
        await page.wait_for_timeout(5000)

        # Step 3: Hover over "Shop by Handset"
        hover_xpath = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[2]/a'
        await page.locator(f'xpath={hover_xpath}').hover()
        await page.wait_for_timeout(1000)  # Give time for dropdown

        # Step 4: Click "Apple iPhone 16 Pro"
        iphone_xpath = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[2]/ul/li/ul/li[2]/ul/li[3]/a'
        await page.locator(f'xpath={iphone_xpath}').click()
        await page.wait_for_load_state('load')

        await load_all_items(page, '//*[@id="facet-browse"]/section/div[3]/div[2]/div[1]/div/div')

        # Step 5: Wait for outer container to load and get count
        await page.wait_for_timeout(2000)  # For lazy loading
        outer_container = page.locator('xpath=//*[@id="facet-browse"]/section/div[3]/div[1]/div[5]')
        await outer_container.wait_for(state="visible")

        outer_divs = outer_container.locator(":scope > div")
        outer_count = await outer_divs.count()

        print(f"Found {outer_count} outer divs")


asyncio.run(test())

In [ ]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

async def load_all_items(page, load_more_xpath):
    while True:
        try:
            # Scroll to the bottom
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
            await asyncio.sleep(2.5)  # give time for lazy loading

            load_more = page.locator(f'xpath={load_more_xpath}')

            if await load_more.is_visible():
                print("Clicking 'Load More'...")
                await load_more.click()
                await asyncio.sleep(1.5)  # wait for new items to load
            else:
                print("No more 'Load More' button. All items loaded.")
                break
        except Exception as e:
            print(f"Error during load more process: {e}")
            break

async def scrape_link(context, href, max_retries=3):
    page = await context.new_page()
    attempt = 0
    while attempt < max_retries:
        try:
            await page.goto(href, timeout=60000, wait_until="load")
            await load_all_items(page, '//*[@id="facet-browse"]/section/div[3]/div[2]/div[1]/div/div')

            item_data = []

            outer_base_xpath = '//*[@id="facet-browse"]/section/div[3]/div[1]/div[5]'
            # await page.locator(f'xpath={outer_base_xpath}').wait_for(state="visible", timeout=5000)
            outer_divs = page.locator(f'xpath={outer_base_xpath}/div')
            outer_count = await outer_divs.count()

            print(f"Found {outer_count} outer divs")

            for i in range(1, outer_count + 1):

                inner_path = f'{outer_base_xpath}/div[{i}]'
                # await page.locator(f'xpath={inner_path}').wait_for(state="visible", timeout=5000)
                inner_divs = page.locator(f'xpath={inner_path}/div')
                inner_count = await inner_divs.count()

                print(f"  Outer div {i} has {inner_count} inner divs (items)")

                for j in range(1, inner_count + 1):
                    try:
                        item_xpath = f'{outer_base_xpath}/div[{i}]/div[{j}]/div/div[2]/span[2]/a/h4'
                        item_name = await page.locator(f'xpath={item_xpath}').inner_text()
                        item_data.append({"group": i, "index": j, "name": item_name})
                    except Exception as e:
                        print(f"    Failed to get phone at outer {i}, inner {j}: {e}")
                        continue

            return {
                "href": href,
                "title": f"{len(item_data)} phones found",
                "phones": item_data
            }

        except Exception as e:
            attempt += 1
            print(f"Attempt {attempt} failed for {href}: {e}")
            await asyncio.sleep(5)
        finally:
            await page.close()
    return {"href": href, "title": "Error after retries", "phones": []}


async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        # Go to the website
        await page.goto("https://shop.alphacomm.com/",wait_until="load")

        # Login
        await page.locator('xpath=//*[@id="login-email"]').fill("Somaan@xclusivetradinginc.com")
        await page.locator('xpath=//*[@id="login-password"]').fill("Metro57398XTI$")
        await page.locator('xpath=//*[@id="LoginRegister.View"]/section/div[2]/div/div/form/fieldset/div[6]/button').click()

        href_links = []

        # Loop through top navigation menus
        for l in range(1, 4):
            top_nav_xpath = f'//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[{l}]/a'
            await page.locator(f'xpath={top_nav_xpath}').hover()
            
            # Wait for dropdown to appear
            mid_ul_xpath = f'//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[{l}]/ul/li/ul'
            try:
                await page.locator(f'xpath={mid_ul_xpath}').wait_for(state='visible', timeout=5000)
            except:
                print(f"No dropdown menu for top nav li[{l}]")
                continue

            # Get all middle li elements
            middle_lis_xpath = f'//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[{l}]/ul/li/ul/li'
            middle_items = await page.locator(f'xpath={middle_lis_xpath}').all()

            for i in range(1, len(middle_items) + 1):
                final_ul_xpath = f'//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[{l}]/ul/li/ul/li[{i}]/ul'

                try:
                    await page.locator(f'xpath={final_ul_xpath}').wait_for(state='attached', timeout=3000)
                    final_links = await page.locator(f'xpath={final_ul_xpath}/li/a').all()

                    for link in final_links:

                        base_url = "https://shop.alphacomm.com"
                        # Inside your for loop where you collect hrefs:
                        href = await link.get_attribute('href')
                        if href:
                            if href.startswith("/"):
                                href = base_url + href
                            href_links.append(href)

                        print(href)
                except:
                    print(f"No final links in li[{i}] of main menu li[{l}]")

        print(f"Total links collected: {len(href_links)}")

        results = []
        batch_size = 5

        for i in range(0, 5, batch_size):
            batch = href_links[i:i+batch_size]
            tasks = [scrape_link(context, href) for href in batch]
            batch_results = await asyncio.gather(*tasks)
            for result in batch_results:
                for phone in result["phones"]:
                    phone["source_url"] = result["href"]  # add the page URL for reference
                    results.append(phone)

        df = pd.DataFrame(results)
        df.to_csv("item_data.csv", index=False)
        print("Saved to item_data.csv")


async def test():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        # Step 1: Open main page
        await page.goto("https://shop.alphacomm.com/", wait_until="load")

        # Step 2: Log in
        await page.locator('xpath=//*[@id="login-email"]').fill("Somaan@xclusivetradinginc.com")
        await page.locator('xpath=//*[@id="login-password"]').fill("Metro57398XTI$")
        await page.locator('xpath=//*[@id="LoginRegister.View"]/section/div[2]/div/div/form/fieldset/div[6]/button').click()
        await page.wait_for_timeout(5000)

        # Step 3: Hover over "Shop by Handset"
        hover_xpath = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[2]/a'
        await page.locator(f'xpath={hover_xpath}').hover()
        await page.wait_for_timeout(1000)  # Give time for dropdown

        # Step 4: Click "Apple iPhone 16 Pro"
        iphone_xpath = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[2]/ul/li/ul/li[2]/ul/li[3]/a'
        await page.locator(f'xpath={iphone_xpath}').click()
        await page.wait_for_load_state('load')

        await load_all_items(page, '//*[@id="facet-browse"]/section/div[3]/div[2]/div[1]/div/div')

        # Step 5: Wait for outer container to load and get count
        await page.wait_for_timeout(2000)  # For lazy loading
        outer_container = page.locator('xpath=//*[@id="facet-browse"]/section/div[3]/div[1]/div[5]')
        await outer_container.wait_for(state="visible")

        outer_divs = outer_container.locator(":scope > div")
        outer_count = await outer_divs.count()

        print(f"Found {outer_count} outer divs")


asyncio.run(test())

In [ ]:
import os
import time
import shutil
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
def wait_for_element_to_load(driver, xpath, timeout=15):
    """Wait for an element to load on the page."""
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.XPATH, xpath))
    )

def wait_for_element_to_be_interactable(driver, xpath, timeout=10):
    """Wait for an element to be present and visible."""
    return WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, xpath))
    )

driver = webdriver.Edge()
wait = WebDriverWait(driver, 15)

# Step 1: Load main page and get to main page
driver.get("https://shop.alphacomm.com/")

wait_for_element_to_be_interactable(driver, f'//*[@id="login-email"]').send_keys("Somaan@xclusivetradinginc.com")
wait_for_element_to_be_interactable(driver, f'//*[@id="login-password"]').send_keys("Metro57398XTI$")
wait_for_element_to_be_interactable(driver, '//*[@id="LoginRegister.View"]/section/div[2]/div/div/form/fieldset/div[6]/button').click()

shop_by_category = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[1]/a'
shop_by_headset = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[2]/a'
shop_by_brand = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[3]/a'

wait_for_element_to_be_interactable(driver,shop_by_category).click()

In [19]:
# Step 2: Find all clickable divs inside the main body

wait_for_element_to_be_interactable(driver,shop_by_category).click()

main_body_xpath = '//*[@id="facet-browse"]/section/div[2]/div[2]'
wait.until(EC.presence_of_element_located((By.XPATH, main_body_xpath)))
main_body = driver.find_element(By.XPATH, main_body_xpath)
clickable_divs = main_body.find_elements(By.XPATH, './div')

for index in range(len(clickable_divs)):
    try:
        # Re-find the div each time to avoid stale reference
        main_body = driver.find_element(By.XPATH, main_body_xpath)
        div = main_body.find_elements(By.XPATH, './div')[index]
        print(div.text)
    except StaleElementReferenceException:
        print(f"Element {index} became stale.")
    except Exception as e:
        print(f"Error with element {index}: {e}")



AUDIO
CHARGING
MEMORY
SCREEN CARE
ATTACHABLES


In [21]:
driver.find_element(By.XPATH, '//*[@id="facet-browse"]/section/div[3]/div[2]/div[1]/div/div').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="facet-browse"]/section/div[3]/div[2]/div[1]/div/div"}
  (Session info: MicrosoftEdge=135.0.3179.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF709A1EE65+16773]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF70997F130+402672]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF709C74BDA+1818314]
	(No symbol) [0x00007FF7097B7B86]
	(No symbol) [0x00007FF7097B7E6B]
	(No symbol) [0x00007FF7097F8C77]
	(No symbol) [0x00007FF7097D8F0F]
	(No symbol) [0x00007FF7097ADF8D]
	(No symbol) [0x00007FF7097F6658]
	(No symbol) [0x00007FF7097D8C33]
	(No symbol) [0x00007FF7097AD4D6]
	(No symbol) [0x00007FF7097ACA82]
	(No symbol) [0x00007FF7097AD303]
	(No symbol) [0x00007FF7098A060D]
	(No symbol) [0x00007FF7098A

In [ ]:
for l in range(1, 4):
    main_button = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[{l}]/a')))

    # Hover to reveal dropdown
    actions = ActionChains(driver)
    actions.move_to_element(main_button).perform()

    # Wait for middle dropdown UL to appear
    wait.until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[{l}]/ul/li/ul')))

    # Get all `li` elements inside the middle ul
    middle_lis_xpath = f'//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[{l}]/ul/li/ul/li'
    middle_lis = driver.find_elements(By.XPATH, middle_lis_xpath)

    # Loop through all of them
    for i in range(1, len(middle_lis) + 1):  # XPath is 1-indexed
        final_list_xpath = f'//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[{l}]/ul/li/ul/li[{i}]/ul'
        try:
            # Wait for this specific sub-list to be present (optional but helpful)
            wait.until(EC.presence_of_element_located((By.XPATH, final_list_xpath)))
            final_links = driver.find_elements(By.XPATH, final_list_xpath + '/li/a')

            for link in final_links:
                print(link.get_attribute('href'))
        except:
            print(f"No final links in li[{i}]")


/Shop-By-Category/Audio/Audio-Adapters
/Shop-By-Category/Audio/In-Ear-Wired
/Shop-By-Category/Audio/In-Ear-Wireless
/Shop-By-Category/Audio/Over-Ear-wireless
/Shop-By-Category/Audio/True-Wireless
/Shop-By-Category/Audio/Speakers
/Shop-By-Category/Charging/cables
/Shop-By-Category/Charging/Car-Chargers
/Shop-By-Category/Charging/Portable-Power
/Shop-By-Category/Charging/Wall-Chargers
/Shop-By-Category/Charging/Wireless-Charging
/Shop-By-Category/Memory/Memory-Cards
/Shop-By-Category/Screen-Care/Liquid-Glass
/Shop-By-Category/Screen-Care/Film
/Shop-By-Category/Screen-Care/Tempered-Glass
/Shop-By-Category/Attachables/AirPod-Cases
/Shop-By-Category/Attachables/Crossbody
/Shop-By-Category/Attachables/Lanyard
/Shop-By-Category/Attachables/Phone-Grips
/Shop-By-Category/Attachables/Mounts
/Shop-By-Category/Attachables/Misc
/Shop-By-Category/Attachables/Watch-Bands
/Shop-by-Handset/Alcatel/Go-Flip-4
/Shop-by-Handset/Alcatel/Alcatel-Joy-Tab-2
/Shop-by-Handset/Apple/iPhone-16e
/Shop-by-Handset/Ap

In [42]:
WebDriverWait(driver, 10).until(EC.element_to_be_clickable(clickable_divs[0])).click()

In [28]:
for div in clickable_divs[:5]:
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(div)).click()
    except StaleElementReferenceException:
        print("Element became stale before it could be clicked.")
    except Exception as e:
        print(f"Unexpected error: {e}")

Element became stale before it could be clicked.
Element became stale before it could be clicked.
Element became stale before it could be clicked.
Element became stale before it could be clicked.
Element became stale before it could be clicked.


In [15]:
# Step 2: Find all clickable divs inside the main body
wait_for_element_to_be_interactable(driver, nagivation_buttons[0]).click()
main_body_xpath = '//*[@id="facet-browse"]/section/div[2]/div[2]'
wait.until(EC.presence_of_element_located((By.XPATH, main_body_xpath)))
main_body = driver.find_element(By.XPATH, main_body_xpath)
clickable_divs = main_body.find_elements(By.XPATH, './div')
clickable_divs

[<selenium.webdriver.remote.webelement.WebElement (session="3585d2879b429db901a39b83456d94d6", element="f.FF2EBACFA117CDAACFBD60ED7A7A97B6.d.D623177C4CA91034AEBE36434DE0326F.e.221")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3585d2879b429db901a39b83456d94d6", element="f.FF2EBACFA117CDAACFBD60ED7A7A97B6.d.D623177C4CA91034AEBE36434DE0326F.e.222")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3585d2879b429db901a39b83456d94d6", element="f.FF2EBACFA117CDAACFBD60ED7A7A97B6.d.D623177C4CA91034AEBE36434DE0326F.e.223")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3585d2879b429db901a39b83456d94d6", element="f.FF2EBACFA117CDAACFBD60ED7A7A97B6.d.D623177C4CA91034AEBE36434DE0326F.e.224")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3585d2879b429db901a39b83456d94d6", element="f.FF2EBACFA117CDAACFBD60ED7A7A97B6.d.D623177C4CA91034AEBE36434DE0326F.e.225")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3585d2879b429db901a3

In [16]:
for index in range(len(clickable_divs)):
    try:
        # Re-find the div each time to avoid stale reference
        main_body = driver.find_element(By.XPATH, main_body_xpath)
        div = main_body.find_elements(By.XPATH, './div')[index]
        print(div.text)
    except StaleElementReferenceException:
        print(f"Element {index} became stale.")
    except Exception as e:
        print(f"Error with element {index}: {e}")

AUDIO
CHARGING
MEMORY
SCREEN CARE
ATTACHABLES
Error with element 5: list index out of range


In [41]:
for item in range(len(nagivation_buttons)): 
    wait_for_element_to_be_interactable(driver, nagivation_buttons[item]).click()

    # Step 2: Find all clickable divs inside the main body
    main_body_xpath = '//*[@id="facet-browse"]/section/div[2]/div[2]'
    wait.until(EC.presence_of_element_located((By.XPATH, main_body_xpath)))
    main_body = driver.find_element(By.XPATH, main_body_xpath)

    clickable_divs = main_body.find_elements(By.XPATH, './div')

    # Step 3: Open each div in a new tab
    main_window = driver.current_window_handle
    for i in range(len(clickable_divs)):
        try:
            # Re-find the clickable divs to avoid stale elements
            main_body = driver.find_element(By.XPATH, main_body_xpath)
            clickable_divs = main_body.find_elements(By.XPATH, './div')

            # Use JavaScript to open link in new tab
            driver.execute_script("window.open(arguments[0].querySelector('a').href, '_blank');", clickable_divs[i])
            time.sleep(1)  # Tiny delay to avoid spamming too fast
        except Exception as e:
            print(f"Error opening div {i}: {e}")

    # Step 4: Now handle all tabs one by one
    all_tabs = driver.window_handles

    for tab in all_tabs:
        if tab == main_window:
            continue  # Skip the main page
        try:
            driver.switch_to.window(tab)

            # Wait for something on the new page
            # Replace 'your_target_xpath_here' with what you want to scrape
            your_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="facet-browse"]/section/div[2]/div[1]/section/div[2]/h1')))
            print(your_element.text)

            driver.close()  # Close tab after work done
        except Exception as e:
            print(f"Error in tab {tab}: {e}")

    # Step 5: Go back to main window
    driver.switch_to.window(main_window)

print("Done with all tabs!")


ATTACHABLES
SCREEN CARE
CHARGING
AUDIO
MEMORY
LG HANDSET
Error in tab 00C09DF526C8069088D815CCD17DE66F: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6A6ABEE65+16773]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF6A6A1F130+402672]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF6A6D14BDA+1818314]
	(No symbol) [0x00007FF6A6857B86]
	(No symbol) [0x00007FF6A6857E6B]
	(No symbol) [0x00007FF6A6898C77]
	(No symbol) [0x00007FF6A6878F0F]
	(No symbol) [0x00007FF6A684DF8D]
	(No symbol) [0x00007FF6A6896658]
	(No symbol) [0x00007FF6A6878C33]
	(No symbol) [0x00007FF6A684D4D6]
	(No symbol) [0x00007FF6A684CA82]
	(No symbol) [0x00007FF6A684D303]
	(No symbol) [0x00007FF6A694060D]
	(No symbol) [0x00007FF6A694D642]
	(No symbol) [0x00007FF6A6945673]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF6A6B99BAA+265882]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF6A6A2C581+457025]
	Microsoft::Applications::Eve